In [70]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
import sys

project_root = "/home/bobby/repos/latent-neural-dynamics-modeling"
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Exploratory Data Analysis


In [72]:
from pathlib import Path
import polars as pl

from utils.file_handling import get_child_subchilds_tuples
from utils.plots import plot_trial_channel, plot_trial_coordinates, plot_tracing_speed

In [73]:
ROOT_PATH = Path(project_root)
DATA_PATH = ROOT_PATH / "resampled_recordings"
PARTICIPANTS_PATH = DATA_PATH / "participants"

In [74]:
def load_participant_session_data(participant_id: str, session_id: str) -> pl.DataFrame:
    p_partition_path = PARTICIPANTS_PATH / participant_id / session_id / "*"
    print(f"Loading data from: {p_partition_path}")
    return pl.read_parquet(p_partition_path)

In [75]:
participants_partitions = get_child_subchilds_tuples(PARTICIPANTS_PATH)
participants_partitions

[('participant_id=PDI4', 'session=3'),
 ('participant_id=PDI4', 'session=2'),
 ('participant_id=PDI1', 'session=4'),
 ('participant_id=PDI1', 'session=2'),
 ('participant_id=PDI3', 'session=4'),
 ('participant_id=PDI3', 'session=2'),
 ('participant_id=PDI2', 'session=4'),
 ('participant_id=PDI2', 'session=2')]

In [76]:
participant_id = "participant_id=PDI1"
session_id = "session=2"

In [77]:
participants = load_participant_session_data(participant_id, session_id)

Loading data from: /home/bobby/repos/latent-neural-dynamics-modeling/resampled_recordings/participants/participant_id=PDI1/session=2/*


In [78]:
from utils.polars import get_trial

In [79]:
trial_df = get_trial(
    participants,
    participant_id="PDI1",
    session=2,
    block=1,
    trial=1,
    columns=[
        "time",
        "LFP_1",
        "chunk_margin",
        "margined_duration",
        "dbs_stim",
        "participant_id",
        "trial",
    ],
    explode=["time", "LFP_1"],
)

In [80]:
plot_trial_channel(
    trial_df,
    channel="LFP_1",
)

In [81]:
trial_df = get_trial(
    participants,
    participant_id="PDI1",
    session=2,
    block=1,
    trial=1,
    columns=[
        "time",
        "ECOG_1",
        "chunk_margin",
        "margined_duration",
        "dbs_stim",
        "participant_id",
        "trial",
    ],
    explode=["time", "ECOG_1"],
)

In [82]:
plot_trial_channel(
    trial_df,
    channel="ECOG_1",
)

* **Visualize Recordings with DBS Markers:**

In [83]:
trial_df = get_trial(
    participants,
    participant_id="PDI1",
    session=2,
    block=1,
    trial=1,
    columns=[
        "motion_time",
        "x",
        "y",
        "chunk_margin",
        "margined_duration",
        "dbs_stim",
        "participant_id",
        "trial",
        "session",
        "block",
    ],
    explode=["motion_time", "x", "y"],
)

In [84]:
plot_trial_coordinates(
    trial_df,
    time="motion_time",
)

In [85]:
plot_trial_coordinates(
    trial_df,
    time="motion_time",
    plot_over_time=True,
)

In [86]:
trial_df = get_trial(
    participants,
    participant_id="PDI1",
    session=2,
    block=1,
    trial=1,
    columns=[
        "time_original",
        "x_interpolated",
        "y_interpolated",
        "chunk_margin",
        "margined_duration",
        "dbs_stim",
        "participant_id",
        "trial",
        "session",
        "block",
    ],
    explode=["time_original", "x_interpolated", "y_interpolated"],
)

In [87]:
plot_trial_coordinates(
    trial_df, x="x_interpolated", y="y_interpolated", time="time_original"
)

In [88]:
plot_trial_coordinates(
    trial_df,
    x="x_interpolated",
    y="y_interpolated",
    time="time_original",
    plot_over_time=True,
)

In [89]:
from utils.polars import read_tsv

In [90]:
sample_participant = "/home/bobby/repos/latent-neural-dynamics-modeling/data/sub-PDI1/ses-2/motion/sub-PDI1_ses-2_task-copydraw_run-01_chunk-01_tracksys-wacom_motion.tsv"

In [91]:
df = read_tsv(sample_participant)

In [92]:
plot_trial_coordinates(df)

### 3. Power Spectral Density (PSD) Analysis (for RQ2)

* **Segment Data:** Separate the preprocessed data into epochs corresponding to DBS ON and DBS OFF conditions.
* **Calculate PSD:**
    - Compute PSD estimates for LFP channels during DBS ON vs. OFF.
    - Compute PSD estimates for ECoG channels during DBS ON vs. OFF.
* **Compare PSDs:**
    - Plot the mean PSD for ON vs. OFF conditions for both LFP and ECoG to identify potential neural markers (e.g., changes in beta-band power).
* **Statistical Testing:**
    - Implement or use a library function for the cluster-based permutation test.
    - Run the test on the PSD data to find significant differences between DBS ON and OFF states.


In [93]:
from utils.ieeg import epoch_trials

In [94]:
participants = participants.with_columns(
    pl.col("LFP_1")
    .map_elements(lambda x: epoch_trials(x), return_dtype=pl.List(pl.List(pl.Float64)))
    .alias("LFP1_epochs")
)

In [95]:
participants_dbs_on = participants.filter(pl.col("dbs_stim") == 1)

In [96]:
participants_dbs_on = participants.filter(pl.col("dbs_stim") == 0)

### 4. Behavioral Data Processing


In [97]:
from utils.motion import tracing_speed

In [98]:
participants = participants.with_columns(
    pl.struct(["x_interpolated", "y_interpolated", "time_original"])
    .map_elements(
        lambda x: tracing_speed(
            x["x_interpolated"], x["y_interpolated"], x["time_original"]
        ),
        return_dtype=pl.List(pl.Float64),
    )
    .alias("tracing_speed")
)

In [99]:
trial_df = get_trial(
    participants,
    participant_id="PDI1",
    session=2,
    block=1,
    trial=1,
    columns=[
        "time_original",
        "tracing_speed",
        "dbs_stim",
        "participant_id",
        "trial",
        "session",
        "block",
    ],
    explode=["time_original", "tracing_speed"],
)

In [100]:
plot_tracing_speed(
    trial_df, tracing_speed="tracing_speed", time="time_original"
)

In [101]:
participants = participants.with_columns(
    pl.struct(["x", "y", "motion_time"])
    .map_elements(
        lambda x: tracing_speed(
            x["x"], x["y"], x["motion_time"]
        ),
        return_dtype=pl.List(pl.Float64),
    )
    .alias("original_tracing_speed")
)

In [102]:
trial_df = get_trial(
    participants,
    participant_id="PDI1",
    session=2,
    block=1,
    trial=1,
    columns=[
        "motion_time",
        "original_tracing_speed",
        "dbs_stim",
        "participant_id",
        "trial",
        "session",
        "block",
    ],
    explode=["motion_time", "original_tracing_speed"],
)

In [103]:
plot_tracing_speed(
    trial_df, tracing_speed="original_tracing_speed", time="motion_time"
)